In [ ]:
%aiida
import os
import sys

import ipywidgets as ipw
from IPython.display import display, clear_output, Image
from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget

IsothermWorkChain = WorkflowFactory('lsmo.isotherm')

In [ ]:
def on_submit():
    output.value = ''
    options = {
            "resources": {
                "num_machines": 1,
                "tot_num_mpiprocs": 1,
                "num_mpiprocs_per_machine": 1,
            },
            "max_wallclock_seconds": 5 * 60 * 60,
            "withmpi": False,
    }
    
    builder = IsothermWorkChain.get_builder()
    
    # Options.
    builder.raspa_base.raspa.metadata.options = options
    builder.zeopp.metadata.options = options

    # Molecule.
    if struct_widget.structure is None:
        output.value = "Specify the framework, please."
        return None
    builder.molecule = Str(molecule.value)
    
    
    # Codes
    if zeopp_code_widget.selected_code is None:
        output.value = "Specify Zeo++ code, please."
        return None

    builder.zeopp.code = zeopp_code_widget.selected_code
    if raspa_code_widget.selected_code is None:
        output.value = "Specify RASPA code, please."
        return None
    builder.raspa_base.raspa.code = raspa_code_widget.selected_code

    # Structure.
    builder.structure = struct_widget.structure_node
    
    # Parameters.
    builder.parameters = Dict(
        dict={  #TODO: create IsothermParameters instead of Dict # pylint: disable=fixme
    "ff_framework": ff_framework.value,  # (str) Forcefield of the structure.
    "ff_separate_interactions": ff_separate_interactions.value,  # (bool) Use "separate_interactions" in the FF builder.
    "ff_mixing_rule": ff_mixing_rule.value,  # (string) Choose 'Lorentz-Berthelot' or 'Jorgensen'.
    "ff_tail_corrections": ff_tail_corrections.value,  # (bool) Apply tail corrections.
    "ff_shifted": ff_shifted.value,  # (bool) Shift or truncate the potential at cutoff.
    "ff_cutoff": ff_cutoff.value,  # (float) CutOff truncation for the VdW interactions (Angstrom).
    "temperature": temperature.value,  # (float) Temperature of the simulation.
    "temperature_list": None,  # (list) To be used by IsothermMultiTempWorkChain.
    "zeopp_volpo_samples": zeopp_volpo_samples.value,  # (int) Number of samples for VOLPO calculation (per UC volume).
    "zeopp_block_samples": zeopp_block_samples.value,  # (int) Number of samples for BLOCK calculation (per A^3).
    "raspa_minKh": 1e16,
    "raspa_widom_cycles": raspa_widom_cycles.value,  # (int) Number of Widom cycles.
})

    return builder

btn_submit = SubmitButtonWidget(IsothermWorkChain,input_dictionary_function=on_submit, disable_after_submit=False, append_output=True)

In [ ]:
# Framework selection.
struct_widget = StructureManagerWidget(
    importers=[
        ("From computer", StructureUploadWidget()),
        ("AiiDA database", StructureBrowserWidget()),
    ],
    storable=False,
    node_class='CifData')

# Codes selection
raspa_code_widget = CodeDropdown(input_plugin='raspa', path_to_root='../../')
zeopp_code_widget = CodeDropdown(input_plugin='zeopp.network', path_to_root='../../')

# Guest molecule and temperature.
temperature = ipw.BoundedIntText(value=400, min=0, max=1e4, description="Temperature:", step=10)
molecule = ipw.Dropdown(
    options=[
        ("CO2", "co2"),
        ("CH4", "ch4"),
        ("N2",  "n2"),
        ("H2O", "h2o"),
        ("H2",  "h2"),
        ("O2",  "o2"),
    ],
    description="Guest molecule",
    style={'description_width':'initial'},
)


# Force field related parameters.
ff_framework = ipw.Dropdown(
    options=["UFF"],
    value='UFF',
    description='Forcefield of the framework:',
    style={'description_width':'initial'},
    disabled=False
)
ff_separate_interactions = ipw.ToggleButtons(
    options=[('Yes', True), ('No', False)],
    description='Separate interactions:',
    #layout={'width':'initial'},
    style={'description_width':'initial'},
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

ff_mixing_rule = ipw.ToggleButtons(
    options=['Lorentz-Berthelot', 'Jorgensen'],
    description='Forcefield mixing rule:',
    style={'description_width':'initial'},
    disabled=False,
    button_style='',
)
ff_tail_corrections = ipw.ToggleButtons(
    options=[('Yes', True), ('No', False)],
    description='Apply tail corrections:',
    style={'description_width':'initial'},
    disabled=False,
    button_style='',
)
ff_shifted = ipw.ToggleButtons(
    options=[('Shift', True), ('Truncate', False)],
    description='Shift or truncate the potential at cutoff:',
    style={'description_width':'initial'},
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
ff_cutoff = ipw.IntText(value=12.0, step=1, description="VdW interactions cutoff:")

ff_box = ipw.VBox([ff_framework, ff_separate_interactions, ff_mixing_rule, ff_shifted])


# Zeo++ settings.
zeopp_volpo_samples = ipw.IntText(value=100000, description="Zeopp volpo samples:", style={'description_width':'initial'})
zeopp_block_samples = ipw.IntText(value=100, description="Number of samples for BLOCK calculation", style={'description_width':'initial'})

zeopp_box = ipw.VBox([zeopp_volpo_samples, zeopp_block_samples])


# Raspa settings.
raspa_widom_cycles = ipw.IntText(value=100000, description="Number of Widom cycles:", style={'description_width':'initial'})

raspa_box = ipw.VBox([raspa_widom_cycles])


# Packing things together.
simulation_accordion = ipw.Accordion([ff_box, zeopp_box, raspa_box])
simulation_accordion.selected_index = None
simulation_accordion.set_title(0, 'Force Field')
simulation_accordion.set_title(1, 'Zeo++ Parameters')
simulation_accordion.set_title(2, 'Raspa Parameters')

output = ipw.HTML("")

In [ ]:
display(
    struct_widget,
    ipw.HBox([molecule, temperature]),
    simulation_accordion,
    ipw.HBox([zeopp_code_widget, raspa_code_widget]),
    btn_submit,
    output
)